<a href="https://colab.research.google.com/github/Pyben-II/Aidigo-logistic/blob/main/Ruben/2_%D0%A2%D1%80%D0%B0%D0%BD%D0%BA%D1%80%D0%B8%D0%B1%D0%B0%D1%86%D0%B8%D1%8F_(SpRecognition_%2BVosk%2BConf).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Установка библиотек

In [ ]:
!pip install pydub # обработка и конвертация аудиофайлов

# Установка библиотек для SpeechRecognition
!pip install SpeechRecognition

# Установка библиотек для vosk
!pip install moviepy                      # работа с видеофайлами
!pip install vosk                         # для использования моделей Vosk для распознавания речи

# Установка библиотек для Conformer
!pip install torchaudio transformers

# Загрузка библиотек

In [ ]:
import os
from pydub import AudioSegment

# Загрузка библиотек для SpeechRecognition
import speech_recognition as sr

# Загрузка библиотек для vosk
from vosk import Model, KaldiRecognizer
from pydub import AudioSegment
import wave                                   # работа с аудиофайлами в формате WAV
import json
# import subprocess                           # Импорт модуля subprocess

# Загрузка библиотек для Conformer
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

In [ ]:
# Загрузка библиотек для Conformer
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

# def для всех

In [ ]:
# Функция для распознавания речи в аудиофайле
def transcribe_spch_recognition(audio_file, txt_folder):
    recognizer = sr.Recognizer()

    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)

        try:
            text = recognizer.recognize_google(audio_data, language="ru-RU")
            print(f'\nТекст из аудиофайла "{wav_file}":')
            print(text)
            # Сохранение текста в отдельный файл
            text_file = os.path.splitext(os.path.basename(audio_file))[0] + ".txt"
            text_path = os.path.join(txt_folder, text_file)
            with open(text_path, "w") as file:
                file.write(text)
        except sr.UnknownValueError:
            print(f'\nТекст из аудиофайла "{wav_file}":')
            print("- Речь не распознана")
        except sr.RequestError as e:
            print(f'\nТекст из аудиофайла "{wav_file}":')
            print("Ошибка при запросе к сервису распознавания: {0}".format(e))

def transcribe_vosk_1(file_path, FRAME_RATE):
    # Открываем аудиофайл с помощью wave
    wf = wave.open(file_path, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    # Чтобы получать слова вместо простых результатов
    rec.SetWords(True)

    # Чтение данных и распознавание
    while True:
        data = wf.readframes(FRAME_RATE)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            print("Процесс распознавания...")  # Логирование процесса

    # Получение итогового результата
    result = rec.FinalResult()
    return result

def transcribe_vosk_2(audio_files, FRAME_RATE, CHANNELS):
    rec = KaldiRecognizer(model, FRAME_RATE)
    rec.SetWords(True)

    # Используя библиотеку pydub делаем предобработку аудио
    mp3 = AudioSegment.from_mp3(audio_files)
    mp3 = mp3.set_channels(CHANNELS)
    mp3 = mp3.set_frame_rate(FRAME_RATE)

    # Преобразуем вывод в json
    rec.AcceptWaveform(mp3.raw_data)
    result = rec.Result()
    text = json.loads(result)["text"]

    return text

def conformer_model(audio_paths):
    # Инициализация процессора и модели Conformer (с переводом на английский)
    # processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    # model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

    # Инициализация процессора и модели Conformer (коверканный русский язык)
    processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-russian")
    model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-russian")

    for audio_path in audio_paths:
        waveform, sample_rate = torchaudio.load(audio_path)
        # waveform, sample_rate = torchaudio.load(audio_path, normalization=True, channels_first=True, sample_rate=16000) # нормализация с частоты дискретизации
        waveform = waveform.mean(dim=0, keepdim=True)
        waveform = waveform.squeeze().numpy()
        waveform = torch.tensor(waveform)


        input_values = processor(waveform, return_tensors="pt", sampling_rate=16000).input_values
        # input_values = processor(waveform, return_tensors="pt", sampling_rate=sample_rate).input_values
        logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)[0]

        print("\nТранскрибация для {}: {}".format(os.path.basename(audio_path), transcription))

# Пути к файлам

In [ ]:
# Для speech_recognition

# Путь к папке с wav файлами
wav_folder = "/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/wav"

# Путь к папке для сохранения текстовых файлов
output_file_path = "/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/text"

# Для VOSK

# Список путей к аудиофайлам
audio_files = ['/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/wav/Microphone_ARU_OFF.wav',
               '/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/wav/лепота.wav',
               '/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/wav/Ларису Ивановну хачу.wav',
               '/content/drive/MyDrive/Пансионат(аудио)/Dataset/Split/wav/audio1_segment_32.wav']

# audio_files = ['/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/wav/Microphone_ARU_OFF.wav',
#                '/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/лепота.mp3',
#                '/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/Ларису Ивановну хачу.mp3',
#                '/content/drive/MyDrive/Пансионат(аудио)/Dataset/Split/audio1_segment_32.mp3']

# Путь к модели vosk
model_path = '/content/drive/MyDrive/Пансионат(аудио)/speech/model_vosk'
# Модель vosk
model = Model(model_path)

# 1_SpeechRecognition

In [ ]:
# Перебор файлов в папке wav
for wav_file in os.listdir(wav_folder):
    if wav_file.endswith(".wav"):
        wav_path = os.path.join(wav_folder, wav_file)
        transcribe_spch_recognition(wav_path, output_file_path)


Текст из аудиофайла "Phone_ARU_OFF.wav":
три нуля Ольга здравствуйте сегодня вызов такси Подскажите пожалуйста сколько будет стоить проезд от проспекта Просвещения до аэропорта Спасибо

Текст из аудиофайла "Microphone_ARU_OFF.wav":
расстояние до микрофона 0,3 м 0,5 м 2 м 3 м 4 м 5 м 6 м 5 м 4 м 3 м 2 м 0,5 м 0,3

Текст из аудиофайла "навстречу эпохе - scanca.net.wav":
мы идём навстречу эпохи когда Социализм в той или иной его конкретный исторически обусловленной форме станет преобладающим общественной системой на Земле

Текст из аудиофайла "члены - scanca.net.wav":
члены политбюро ЦК КПСС кандидаты в члены политбюро и секретари ЦК КПСС первые секретари ЦК Компартии союзных республик другие члены Президиума торжественного заседания

Текст из аудиофайла "с добрым утром - scanca.net.wav":
начинаем нашу воскресную радиопередачу с добрым утром

Текст из аудиофайла "Санта Барбара.wav":
- Речь не распознана

Текст из аудиофайла "мерлин монро - scanca.net.wav":
И вообще я Мэрилин Монро У меня

# 2_VOSK

## код 1

In [ ]:
# Устанавливаем Frame Rate
FRAME_RATE = 8000

for audio_file in audio_files:
    text = transcribe_vosk_1(audio_file, FRAME_RATE)
    output_file_path = "/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/text/vosk1_" + os.path.basename(audio_file).replace('.wav', '.txt')
    # output_file_path = "/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/text/vosk1_" + str(audio_files.index(audio_file_path) + 1) + ".txt"

    with open(output_file_path, "w") as text_file:
        print(f"\nТекст в файле: '{os.path.basename(output_file_path)}'.")
        print(f"- '{text}'.")
        text_file.write(text)
        print(f"Текст успешно сохранен в файл '{output_file_path}'.")

Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...
Процесс распознавания...

Текст в файле: 'vosk1_Microphone_ARU_OFF.txt'.
- '{
  "result" : [{
      "conf" : 1.000000,
      "end" : 1.590000,
      "start" : 0.900000,
      "word" : "расстояние"
    }, {
      "conf" : 1.000000,
      "end" : 1.710000,
      "start" : 1.590000,
      "word" : "до"
    }, {
      "conf" : 1.000000,
      "end" : 2

## код 2(с каналом)

In [ ]:
# Устанавливаем Frame Rate
FRAME_RATE = 8000
CHANNELS=1

for audio_file in audio_files:
    transcribed_text = transcribe_vosk_2(audio_file, FRAME_RATE, CHANNELS)

    output_file_path = "/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/text/vosk2_" + os.path.basename(audio_file).replace('.wav', '.txt')

    with open(output_file_path, 'w') as f:
        json.dump(transcribed_text, f, ensure_ascii=False, indent=4)
        print(f"\nВ файле '{os.path.basename(output_file_path)}':")
        print(f"- '{transcribed_text}'.")
        print(f"Текст сохранен в файл: '{output_file_path}'.")

print("\nВсе аудиофайлы успешно обработаны.")


В файле 'vosk2_Microphone_ARU_OFF.txt':
- 'расстояние до микрофона ноль три метра ноль пять метр два метра три метра четыре метра пять метров шесть метров пять метров четыре метра три метра два метра метр ноль пять метра ноль три'.
Текст сохранен в файл: '/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/text/vosk2_Microphone_ARU_OFF.txt'.

В файле 'vosk2_лепота.txt':
- ''.
Текст сохранен в файл: '/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/text/vosk2_лепота.txt'.

В файле 'vosk2_Ларису Ивановну хачу.txt':
- 'алло лариса'.
Текст сохранен в файл: '/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/text/vosk2_Ларису Ивановну хачу.txt'.

В файле 'vosk2_audio1_segment_32.txt':
- 'если инфекция'.
Текст сохранен в файл: '/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/text/vosk2_audio1_segment_32.txt'.

Все аудиофайлы успешно обработаны.


# 3_Conformer

In [ ]:
# Path to the folder containing audio files
wav_folder = "/content/drive/MyDrive/Пансионат(аудио)/Dataset/Короткие фразы/wav"
audio_files = [os.path.join(wav_folder, f) for f in os.listdir(wav_folder) if f.endswith('.wav')]

conformer_model(audio_files)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT e


Транскрибация для Phone_ARU_OFF.wav: примененнынат здрастиэте взаь таксим да поскажите пожалуйста сколько будет муде проестот проскекты посвещения ду эрапорта гоумира ра второенанать поаза нанепоет его нетоботнетамен посибо

Транскрибация для Microphone_ARU_OFF.wav: рассталунита никрофона ноль телеметра но пят нет два нетр нмета ни метр пятнет чсё зната  птната чутоне метра  метр два нтра нет а пятмета альтри
